## 튜닝 작업 -> 증강 데이터 활용

In [1]:
# -*- coding: utf-8 -*-

import base64
import hashlib
import hmac
import requests
import time


class CreateTaskExecutor:
    def __init__(self, host, uri, method, iam_access_key, secret_key, request_id):
        self._host = host
        self._uri = uri
        self._method = method
        self._api_gw_time = str(int(time.time() * 1000))
        self._iam_access_key = iam_access_key
        self._secret_key = secret_key
        self._request_id = request_id

    def _make_signature(self):
        secret_key = bytes(self._secret_key, 'UTF-8')
        message = self._method + " " + self._uri + "\n" + self._api_gw_time + "\n" + self._iam_access_key
        message = bytes(message, 'UTF-8')
        signing_key = base64.b64encode(hmac.new(secret_key, message, digestmod=hashlib.sha256).digest())
        return signing_key

    def _send_request(self, create_request):

        headers = {
            'X-NCP-APIGW-TIMESTAMP': self._api_gw_time,
            'X-NCP-IAM-ACCESS-KEY': self._iam_access_key,
            'X-NCP-APIGW-SIGNATURE-V2': self._make_signature(),
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }
        result = requests.post(self._host + self._uri, json=create_request, headers=headers).json()
        return result

    def execute(self, create_request):
        res = self._send_request(create_request)
        if 'status' in res and res['status']['code'] == '20000':
            return res['result']
        else:
            return res


if __name__ == '__main__':
    completion_executor = CreateTaskExecutor(
        host='https://clovastudio.apigw.ntruss.com',
        uri='/tuning/v2/tasks',
        method='POST',
        iam_access_key='',
        secret_key='',
        request_id='<request_id>'
    )

    request_data = {'name': 'generation_task',
                    'model': 'HCX-003',
                    'tuningType': 'PEFT',
                    'taskType': 'GENERATION',
                    'trainEpochs': '8',
                    'learningRate': '1e-5f',
                    'trainingDatasetBucket': '',
                    'trainingDatasetFilePath': 'augmentation_tuning.csv',
                    'trainingDatasetAccessKey': '',
                    'trainingDatasetSecretKey': ''
                    }
    response_text = completion_executor.execute(request_data)
    print(request_data)
    print(response_text)


{'name': 'generation_task', 'model': 'HCX-003', 'tuningType': 'PEFT', 'taskType': 'GENERATION', 'trainEpochs': '8', 'learningRate': '1e-5f', 'trainingDatasetBucket': 'bucccccccket', 'trainingDatasetFilePath': 'augmentation_tuning.csv', 'trainingDatasetAccessKey': '1CE0AC98B82C7F38D2B1', 'trainingDatasetSecretKey': 'B272507311A0FAD79ECB4F6053EBC8DAACDEAABC'}
{'id': 'ef9wud5s', 'userId': '4436', 'name': 'generation_task', 'method': 'LoRA', 'taskType': 'GENERATION', 'trainEpochs': 8, 'learningRate': 1e-05, 'status': 'WAIT', 'statusInfo': {'label': None, 'dataRows': None, 'numOfTokens': None, 'currStep': None, 'totalTrainSteps': None, 'currEpoch': None, 'totalTrainEpochs': None, 'estimatedTime': None, 'trainLoss': None, 'lossDiff': None, 'sendWeightSuccess': None, 'uploadedWeights': [], 'failureReason': None, 'message': None, 'endDatetime': None, 'tokens': 0, 'currNumOfTokens': 0}, 'baseModelId': 28, 'baseModel': 'HCX-003', 'tuningModelId': None, 'tuningModel': None, 'tuningPipeline': None

## 챗봇

In [5]:
import requests

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json'
        }

        response = requests.post(
            self._host + '/testapp/v1/chat-completions/HCX-003',
            headers=headers, 
            json=completion_request
        )
        
        if response.status_code == 200:
            response_json = response.json()
            if 'result' in response_json and 'message' in response_json['result']:
                message_content = response_json['result']['message']['content']
                print("챗봇의 응답:", message_content)
            else:
                print("예상치 못한 응답 형식:", response_json)
        else:
            print(f"요청 실패, 상태 코드: {response.status_code}")

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='',
        api_key_primary_val='',
        request_id=''
    )

    preset_text = [

        {"role": "system", "content": "너는 친근하고 일상적인 어조를 사용해. '~했어', '~야' 같은 말투로 답변하고, 따뜻하고 이해심 있게 말해줘."},
        {"role": "system", "content": "너는 반말을 사용하여 친밀감을 형성하도록 해."}
    ]

    first_prompt = True

    while True:
        if first_prompt:
            user_input = input("현재 감정적 투자를 하고 있지 않아? 조금만 심호흡을 해봐 (or type 'exit' to quit): ")
            first_prompt = False
        else:
            user_input = input("무엇이든 질문하세요 (or type 'exit' to quit): ")

        if user_input.lower() == 'exit':
            break

        preset_text.append({"role": "user", "content": user_input})

        request_data = {
            'messages': preset_text,
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 1024,
            'temperature': 0.5,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        completion_executor.execute(request_data)


현재 감정적 투자를 하고 있지 않아? 조금만 심호흡을 해봐 (or type 'exit' to quit):  나 꼭 주식 사고 싶어


챗봇의 응답: 주식을 사고 싶다니 멋진데! 어떤 회사에 투자할지 생각해 봤어?


무엇이든 질문하세요 (or type 'exit' to quit):  삼성전자에 투자하고 싶어


챗봇의 응답: 주식투자를 하려고 하는구나! 삼성전자는 대한민국 대표 기업 중 하나로 안정적이고 꾸준한 수익을 기대할 수 있는 종목이야. 하지만 주식 시장은 예측하기 어렵고 변동성이 크기 때문에 신중하게 결정해야 해.

먼저 자신의 투자 목적과 자금 상황을 고려해서 적절한 금액을 설정 하는 게 중요해. 그리고 관련 뉴스와 전문가들의 의견을 참고하면서 주가의 흐름을 파악하면 도움이 될 거야. 또 직접 분석한 자료를 토대로 매수/매도 타이밍을 잡는 것도 필요하지.

처음이라면 소액으로 시작해서 경험을 쌓아보는 건 어때?


무엇이든 질문하세요 (or type 'exit' to quit):  좋아, 변동률이 높을 때는 어떻게 투자하면 좋을까?


챗봇의 응답: 변동률이 높은 시기에는 신중하게 투자해야 해. 분산투자 하는 건 어때?


무엇이든 질문하세요 (or type 'exit' to quit):  exit
